In [9]:
from enum import Enum
from datetime import datetime
import sqlite3

con = sqlite3.connect("movies.db")
cur = con.cursor()

class Genre(Enum):
    SCI_FI = "Sci-Fi"
    ACTION = "Action"
    ADVENTURE = "Adventure"

    @classmethod
    def valueTuples(cls):
        return [(e.value,) for e in cls]

    @classmethod
    def createTable(cls):
        cur.execute('CREATE TABLE IF NOT EXISTS genres (genre TEXT PRIMARY KEY)')
        cur.executemany('INSERT INTO genres VALUES(?)', cls.valueTuples())
        con.commit()

    @classmethod
    def removeTable(cls):
        cur.execute('DROP TABLE IF EXISTS genres')
        con.commit()

    @classmethod
    def getAll(cls):
        cur.execute('SELECT * FROM genres')
        sqlResult = cur.fetchall()
        return [Genre(p[0]) for p in sqlResult]

class Country(Enum):
    USA = 'USA'
    UK = 'UK'
    INDIA = 'India'
    FRANCE = 'France'
    GERMANY = 'Germany'

    @classmethod
    def valueTuples(cls):
        return [(e.value,) for e in cls]

    @classmethod
    def createTable(cls):
        cur.execute('CREATE TABLE IF NOT EXISTS countries (country TEXT PRIMARY KEY)')
        cur.executemany('INSERT INTO countries VALUES(?)', cls.valueTuples())
        con.commit()

    @classmethod
    def removeTable(cls):
        cur.execute('DROP TABLE IF EXISTS countries')
        con.commit()

    @classmethod
    def getAll(cls):
        cur.execute('SELECT * FROM countries')
        sqlResult = cur.fetchall()
        return [Country(p[0]) for p in sqlResult]

class Director:
    def __init__(self, name, birth_year, country):
        self.name = name
        self.birth_year = birth_year
        self.country = country

    @classmethod
    def createTable(cls):
        cur.execute('''CREATE TABLE IF NOT EXISTS directors (
                        name TEXT PRIMARY KEY,
                        birth_year INTEGER,
                        country TEXT,
                        FOREIGN KEY(country) REFERENCES countries(country))''')
        con.commit()

    def insert(self):
        name_str = str(self.name)
        birth_year_str = str(self.birth_year)
        country_str = str(self.country)

        cur.execute('''INSERT INTO directors VALUES(?,?,?)''', (name_str, birth_year_str, country_str))
        con.commit()

    def update(self):
        cur.execute('''UPDATE directors SET birth_year=?, country=? WHERE name=?''', (self.birth_year, self.country, self.name))
        con.commit()

    @classmethod
    def get(cls, name):
        cur.execute('SELECT birth_year, country FROM directors WHERE name=?', (name,))
        res = cur.fetchone()
        return Director(name, res[0], res[1])

    @classmethod
    def getAll(cls):
        cur.execute('SELECT name, birth_year, country FROM directors')
        res = cur.fetchall()
        return [Director(el[0], el[1], el[2]) for el in res]

    def __str__(self):
        return "(%s, %s, %s)" % (self.name, self.birth_year, self.country)

class Movie:
    def __init__(self, title, release_date, director, genre, reception=None):
        self.title = title
        self.release_date = release_date
        self.director = director
        self.genre = genre
        self.reception = reception

    @classmethod
    def createTable(cls):
        cur.execute('''CREATE TABLE IF NOT EXISTS movies (
                        title TEXT PRIMARY KEY,
                        release_date TEXT,
                        director TEXT,
                        genre TEXT,
                        reception REAL,
                        FOREIGN KEY(director) REFERENCES directors(name),
                        FOREIGN KEY(genre) REFERENCES genres(genre))''')
        con.commit()

    def insert(self):
        cur.execute('''INSERT INTO movies VALUES(?,?,?,?,?)''', (self.title, self.release_date, self.director.name, self.genre.value, self.reception))
        con.commit()

    def update(self):
        cur.execute('''UPDATE movies SET release_date=?, director=?, genre=?, reception=? WHERE title=?''', (self.release_date, self.director.name, self.genre.value, self.reception, self.title))
        con.commit()

    @classmethod
    def get(cls, title):
        cur.execute('''SELECT release_date, director, genre, reception FROM movies WHERE title=?''', (title,))
        res = cur.fetchone()
        director = Director.get(res[1]) if res[1] else None
        genre = Genre(res[2])
        return Movie(title, res[0], director, genre, res[3])

    def __str__(self):
        return "(%s, %s, %s, %s, %s)" % (self.title, self.release_date, self.director.name, self.genre.value, self.reception)
cur.execute('DROP TABLE IF EXISTS genres')
con.commit()

cur.execute('DROP TABLE IF EXISTS countries')
con.commit()

cur.execute('DROP TABLE IF EXISTS directors')
con.commit()

cur.execute('DROP TABLE IF EXISTS movies')
con.commit()

Genre.createTable()
Country.createTable()
Director.createTable()
Movie.createTable()

genre1 = Genre.SCI_FI
genre2 = Genre.ACTION
country1 = Country.USA
country2 = Country.UK
director1 = Director("Neil Burger", 1971, country1)
director2 = Director("Another Director", 1985, country2)
movie1 = Movie("Divergent", "March 21, 2014", director1, genre1, 75)
movie2 = Movie("Another Movie", "Some Release Date", director2, genre2)

director1.insert()
director2.insert()
movie1.insert()
movie2.insert()

print(Movie.get("Divergent"))
print(Movie.get("Another Movie"))


(Divergent, March 21, 2014, Neil Burger, Sci-Fi, 75.0)
(Another Movie, Some Release Date, Another Director, Action, None)
